<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i> 유방암 데이터를 통한 앙상블_부스팅 실습
  </h2>
</div>


In [ ]:
from sklearn.metrics import confusion_matrix # 오차행렬
from sklearn.metrics import accuracy_score # 정확도
from sklearn.metrics import precision_score # 정밀도
from sklearn.metrics import recall_score # 재현율
from sklearn.metrics import roc_auc_score # auc score
from sklearn.metrics import f1_score 

def get_clf_eval(y_test,y_pred):
    matrix = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    auc = roc_auc_score(y_test,y_pred)
    
    print('{}'.format('RESULT'))
    print('Confusion Matrix : ')
    print(matrix)
    print('-'*30)
    
    print('Accuaracy Score : ')
    print(accuracy)
    print('-'*30)
    
    print('Precision Score : ')
    print(precision)
    print('-'*30)
    
    print('Recall Score : ')
    print(recall)
    print('-'*30)
    
    print('F1 Score : ')
    print(f1)
    print('-'*30)
    
    print('AUC Score : ')
    print(auc)
    print('-'*30)

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# mpl.rcParams['font.family'] = 'AppleGothic'

In [2]:
cancer = load_breast_cancer() # 데이터를 불러옴 
# 데이터를 프레임으로 만듬 
cancer_df = pd.DataFrame(data=cancer.data,
                         columns=cancer.feature_names)
# Y label == target 
cancer_df['target'] = cancer.target
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


---

## Boosting

- 여러 부스팅 알고리즘


```python 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
```

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  GradientBoostingClassifier
  </h2>
</div>


In [27]:
from sklearn.ensemble import GradientBoostingClassifier
gbm_clf = GradientBoostingClassifier()

In [28]:
gbm_clf.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [29]:
gbm_clf.score(X_test,y_test)

0.9532163742690059

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  XGBClassifier
  </h2>
</div>


In [35]:
from xgboost import XGBClassifier

In [36]:
xgb_clf = XGBClassifier()

In [37]:
xgb_clf.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [38]:
xgb_clf.score(X_test,y_test)

0.9649122807017544

In [ ]:
xgb_pred = xgb_clf.predict(X_test)

In [ ]:
get_clf_eval(y_test,xgb_pred)

In [39]:
# 조기 중단 기능 (early stopping) 활용
eval_data = [(X_test,y_test)]
xgb_clf = XGBClassifier(n_estimators=500)
xgb_clf.fit(X_train,y_train,
            early_stopping_rounds= 100,
            eval_set = eval_data,
            eval_metric = 'logloss')

[0]	validation_0-logloss:0.615593
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.553036
[2]	validation_0-logloss:0.500498
[3]	validation_0-logloss:0.453844
[4]	validation_0-logloss:0.415334
[5]	validation_0-logloss:0.381626
[6]	validation_0-logloss:0.354083
[7]	validation_0-logloss:0.326348
[8]	validation_0-logloss:0.304611
[9]	validation_0-logloss:0.284143
[10]	validation_0-logloss:0.267726
[11]	validation_0-logloss:0.252263
[12]	validation_0-logloss:0.239003
[13]	validation_0-logloss:0.225695
[14]	validation_0-logloss:0.214112
[15]	validation_0-logloss:0.203558
[16]	validation_0-logloss:0.194984
[17]	validation_0-logloss:0.186837
[18]	validation_0-logloss:0.179552
[19]	validation_0-logloss:0.172228
[20]	validation_0-logloss:0.16536
[21]	validation_0-logloss:0.159196
[22]	validation_0-logloss:0.153946
[23]	validation_0-logloss:0.14869
[24]	validation_0-logloss:0.144637
[25]	validation_0-logloss:0.141697
[26]	validation_0-logloss:0.13893

[230]	validation_0-logloss:0.06704
[231]	validation_0-logloss:0.066928
[232]	validation_0-logloss:0.066729
[233]	validation_0-logloss:0.066932
[234]	validation_0-logloss:0.066685
[235]	validation_0-logloss:0.066727
[236]	validation_0-logloss:0.066824
[237]	validation_0-logloss:0.066735
[238]	validation_0-logloss:0.066669
[239]	validation_0-logloss:0.066866
[240]	validation_0-logloss:0.066625
[241]	validation_0-logloss:0.066539
[242]	validation_0-logloss:0.066399
[243]	validation_0-logloss:0.066496
[244]	validation_0-logloss:0.066305
[245]	validation_0-logloss:0.066331
[246]	validation_0-logloss:0.066228
[247]	validation_0-logloss:0.066147
[248]	validation_0-logloss:0.066084
[249]	validation_0-logloss:0.066278
[250]	validation_0-logloss:0.066048
[251]	validation_0-logloss:0.066238
[252]	validation_0-logloss:0.066051
[253]	validation_0-logloss:0.065917
[254]	validation_0-logloss:0.066012
[255]	validation_0-logloss:0.065933
[256]	validation_0-logloss:0.065804
[257]	validation_0-logloss:0.

[459]	validation_0-logloss:0.062029
[460]	validation_0-logloss:0.061918
[461]	validation_0-logloss:0.061769
[462]	validation_0-logloss:0.061785
[463]	validation_0-logloss:0.061896
[464]	validation_0-logloss:0.061787
[465]	validation_0-logloss:0.061864
[466]	validation_0-logloss:0.061757
[467]	validation_0-logloss:0.061834
[468]	validation_0-logloss:0.061728
[469]	validation_0-logloss:0.061837
[470]	validation_0-logloss:0.06174
[471]	validation_0-logloss:0.061749
[472]	validation_0-logloss:0.061826
[473]	validation_0-logloss:0.061722
[474]	validation_0-logloss:0.061799
[475]	validation_0-logloss:0.061696
[476]	validation_0-logloss:0.061807
[477]	validation_0-logloss:0.061823
[478]	validation_0-logloss:0.061728
[479]	validation_0-logloss:0.061627
[480]	validation_0-logloss:0.061703
[481]	validation_0-logloss:0.061812
[482]	validation_0-logloss:0.061709
[483]	validation_0-logloss:0.061649
[484]	validation_0-logloss:0.061659
[485]	validation_0-logloss:0.061734
[486]	validation_0-logloss:0.

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  AdaBoostClassifier
  </h2>
</div>


<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  LGBMClassifier
  </h2>
</div>
